In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# install packages needed for TimeSformer
! pip install torchvision
! pip install 'git+https://github.com/facebookresearch/fvcore'
! pip install simplejson
! pip install einops
! pip install timm
! pip install psutil
! pip install scikit-learn
! pip install opencv-python
! pip install tensorboard
! pip install av

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 71.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 37.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 90.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
! git clone https://github.com/yiyixuxu/TimeSformer.git

Cloning into 'TimeSformer'...
remote: Enumerating objects: 503, done.
remote: Counting objects: 100% (165/165), done.
remote: Compressing objects: 100% (75/75), done.
remote: Total 503 (delta 117), reused 90 (delta 90), pack-reused 338 (from 1)
Receiving objects: 100% (503/503), 2.89 MiB | 29.31 MiB/s, done.
Resolving deltas: 100% (200/200), done.


In [ ]:
%cd TimeSformer/

/content/TimeSformer


In [ ]:
import torch
import cv2
from PIL import Image
import os
from torchvision import transforms
from timesformer.models.vit import TimeSformer

# === Configuration ===
clip_size = 16
class_names = ['Fight', 'NonFight']  # Update as needed

# === Transform ===
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.45, 0.45, 0.45], std=[0.225, 0.225, 0.225])
])

# === Load Pre-trained Model ===
model = TimeSformer(
    img_size=224,
    num_classes=2,
    num_frames=clip_size,
    attention_type='divided_space_time'
).cuda()

checkpoint = torch.load('/content/drive/MyDrive/timesformer/pretrained/final.pth')
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# === Helper to Predict a Clip ===
def predict_clip(frames):
    # Stack the frames and apply transformations
    transformed = torch.stack([
        transform(Image.fromarray(cv2.cvtColor(f, cv2.COLOR_BGR2RGB)))  # (3, 224, 224)
        for f in frames
    ])  # (16, 3, 224, 224)

    # Permute the tensor to match the model's expected input: (1, 16, 3, 224, 224)
    transformed = transformed.permute(1, 0, 2, 3)  # (3, 16, 224, 224) -> (1, 16, 3, 224, 224)
    transformed = transformed.unsqueeze(0)  # Add batch dimension -> (1, 16, 3, 224, 224)

    # Move to the correct device (GPU or CPU)
    transformed = transformed.to(device)

    with torch.no_grad():
        # Forward pass through the model
        logits = model(transformed)  # (1, num_classes)
        probs = torch.softmax(logits, dim=-1)  # (1, num_classes)
        pred_class = torch.argmax(probs, dim=-1).item()

    return class_names[pred_class], probs.squeeze().cpu()



# === Main Processing ===
def process_video(video_path, output_path):
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'XVID'), fps, (w, h))

    frames = []
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frames.append(frame)

        if len(frames) == clip_size:
            label, _ = predict_clip(frames)
            print(f"Prediction: {label}")
            for f in frames:
                cv2.putText(f, label, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                out.write(f)
            frames = []  # Reset for next 16-frame chunk

    cap.release()
    out.release()
    print(f"Inference video saved to: {output_path}")

# === Example Usage ===
process_video('/content/drive/MyDrive/demo.avi', '/content/drive/MyDrive/demo_output.avi')


Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-vitjx/jx_vit_base_p16_224-80ecf9dd.pth" to /root/.cache/torch/hub/checkpoints/jx_vit_base_p16_224-80ecf9dd.pth


Prediction: NonFight
Prediction: NonFight
Prediction: NonFight
Prediction: NonFight
Prediction: NonFight
Prediction: NonFight
Prediction: NonFight
Prediction: NonFight
Prediction: NonFight
Prediction: NonFight
Prediction: NonFight
Prediction: NonFight
Prediction: NonFight
Prediction: NonFight
Prediction: NonFight
Prediction: NonFight
Prediction: NonFight
Prediction: NonFight
Prediction: Fight
Prediction: Fight
Prediction: Fight
Prediction: Fight
Prediction: Fight
Prediction: Fight
Prediction: Fight
Prediction: Fight
Prediction: Fight
Prediction: Fight
Prediction: Fight
Prediction: Fight
Prediction: Fight
Prediction: Fight
Prediction: Fight
Prediction: Fight
Prediction: Fight
Prediction: Fight
Prediction: Fight
Prediction: Fight
Prediction: Fight
Prediction: Fight
Prediction: Fight
Prediction: Fight
Prediction: Fight
Prediction: Fight
Prediction: Fight
Prediction: Fight
Prediction: Fight
Prediction: Fight
Prediction: Fight
Prediction: Fight
Prediction: Fight
Prediction: Fight
Prediction